**Import Library**

In [8]:
import re
import pandas as pd
import nltk
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import emoji
from nltk.tokenize import RegexpTokenizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

<h1>Baca Data<h1>

In [9]:
df = pd.read_csv("data.csv")
texts = df["full_text"].astype(str)
texts.to_csv("data.csv", index=False)

<h1>PraPemrosesan import Stemmer dari Sastrawi</h1>

In [10]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_nltk = set(stopwords.words('indonesian'))
stop_sastrawi = set(StopWordRemoverFactory().get_stop_words())
stop_words = stop_nltk.union(stop_sastrawi)


tokenizer = RegexpTokenizer(r'\w+')

Stemmer dari Sastrawi untuk mereduksi kata-kata Indonesia ke bentuk dasar. Kode menggabungkan daftar stopwords dari NLTK dan Sastrawi ke dalam satu himpunan stop_words untuk memfilter kata-kata umum. Selain itu, RegexpTokenizer dengan pola r'\w+' digunakan untuk memisahkan teks menjadi token alfanumerik, menghilangkan tanda baca. Pengaturan ini mendukung pra-pemrosesan teks untuk analisis seperti sentimen

<h1>Pembersihan data menggunakan case folding CPMK 2<h1>

In [11]:
def case_folding(text):
    text = text.lower()
    return text

df_new = pd.DataFrame({
    "full_text": texts,
    "case_folding": texts.apply(case_folding)
})

df_new.to_csv("data_casefolding.csv", index=False)

Mengubah teks menjadi huruf kecil untuk standardisasi. Fungsi tersebut diterapkan pada daftar teks dalam variabel texts untuk membuat kolom baru bernama case_folding. Kode kemudian membuat DataFrame baru df_new menggunakan pandas, yang berisi kolom full_text (teks asli) dan case_folding (teks setelah konversi huruf kecil). Terakhir, DataFrame ini disimpan ke file CSV bernama data_casefolding.csv

<h1>Preprocessing regex, tokenisasi dan pemakaian stemmer CPMK 3<h1>

In [12]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', ' ', text)
    text = re.sub(r'@\w+', ' ', text)
    text = emoji.demojize(text)
    text = re.sub(r'\d+',' ', text)
    text = re.sub(r'[^\w\s]',' ', text)
    text = re.sub(r'(.)\1{2,}', r'\1', text)
    text = re.sub(r'\s+',' ', text).strip()
    tokens = tokenizer.tokenize(text)
    tokens = [w for w in tokens if w not in stop_words]
    tokens = [stemmer.stem(w) for w in tokens]

    return " ".join(tokens)

df["preprocessing"] = texts.apply(clean_text)

df_new = pd.DataFrame({
    "full_text": texts,
    "case_folding CPMK2": texts.apply(case_folding),
    "preprocessing CPMK3": df["preprocessing"]
})

df_new.to_csv("data_prepocessing.csv", index=False)

Preprocessing teks dengan fungsi clean_text untuk membersihkan dan menstandarisasi data teks agar lebih mudah dianalisis. Proses ini melibatkan beberapa langkah: mengubah teks menjadi huruf kecil (case folding), menghapus URL (dengan pola http\S+|www\S+), menghapus mention seperti @username, mengonversi emoji menjadi representasi teks menggunakan emoji.demojize, menghapus angka, menghapus tanda baca dan karakter khusus, mengurangi karakter berulang (misalnya "aaa" menjadi "a"), serta menghapus spasi berlebih untuk menghasilkan teks yang bersih dan terstandar. Hasilnya adalah teks yang lebih konsisten dan siap untuk tahap analisis berikutnya, seperti tokenisasi.

Tokenisasi dengan memanggil tokenizer.tokenize(text), yang memecah teks yang telah dibersihkan menjadi unit-unit kecil yang disebut token (biasanya kata-kata individual). Proses ini penting karena memungkinkan analisis teks pada tingkat kata, bukan sebagai satu string panjang. Tokenisasi memisahkan teks berdasarkan spasi atau aturan tertentu yang ditentukan oleh tokenizer yang digunakan (misalnya, library seperti NLTK atau spaCy). Tokenisasi menghasilkan daftar kata-kata yang kemudian akan diproses lebih lanjut, seperti menghapus stop words dan melakukan stemming untuk analisis teks yang lebih mendalam.

Stemming dilakukan dengan menggunakan stemmer.stem(w) pada setiap token setelah tokenisasi dan penghapusan stop words. Stemming adalah proses mengurangi kata ke bentuk dasarnya atau akarnya (stem) dengan menghapus imbuhan, seperti mengubah "berlari" menjadi "lari" atau "running" menjadi "run". Tujuannya adalah untuk menyatukan variasi kata yang memiliki makna serupa agar analisis teks lebih efisien, misalnya dalam text mining atau klasifikasi teks. Stemming diterapkan pada setiap token untuk menghasilkan daftar kata-kata dalam bentuk dasar yang kemudian digabung kembali menjadi string dengan spasi sebagai pemisah untuk disimpan dalam kolom "preprocessing".